# Ejercicio AvCont2 (b) - Codificación de Huffman

* Óscar Sementé Solà
* Abdelkarim Azzouguagh Ouniri
* Rodrigo Cabezas Quirós

### [Apartado extra] Algoritmo generación tablas de probabilidades de símbolos

In [1]:
def huffman_tables(e):
    """
    Generación de las tablas de probabilidades de símbolos con Huffman
    """
    simbols = list(e.keys())
    probs, iters = [(k, v/100) for k, v in sorted(e.items(), key=lambda x: x[1], reverse=True)], []
    # Construye los valores del árbol, en caso de que haya más de un símbolo de entrada
    while True:
        # En cada iteración, se ordenan los símbolos en función de su probabilidad (orden descendente)
        probs = sorted(probs, key=lambda x: x[1], reverse=True)
        # Y por cada iteración se guarda el valor de la tabla
        iters.append(probs.copy())
        
        # Condición de salida, todos los símbolos juntos y probabilidad final (1 -> 100%) 
        if len(probs) == 1:
            break
            
        # Sumar probabilidades, juntar símbolos y añadirlos a la lista
        apos, lpos = probs.pop(-2), probs.pop(-1)
        nprob, nsim = apos[1] + lpos[1], apos[0] + lpos[0]
        probs.append((nsim, round(nprob, 3)))
        
    return iters
            
data = {"D": 30, "K": 20, "Q": 20, "J": 15, "10": 10, "9": 5}
huffman_tables(data)

[[('D', 0.3), ('K', 0.2), ('Q', 0.2), ('J', 0.15), ('10', 0.1), ('9', 0.05)],
 [('D', 0.3), ('K', 0.2), ('Q', 0.2), ('J', 0.15), ('109', 0.15)],
 [('D', 0.3), ('J109', 0.3), ('K', 0.2), ('Q', 0.2)],
 [('KQ', 0.4), ('D', 0.3), ('J109', 0.3)],
 [('DJ109', 0.6), ('KQ', 0.4)],
 [('DJ109KQ', 1.0)]]

### a) Implementación del codificador de Huffman.

In [2]:
import math

class HuffmanCoderA:
    
    def __init__(self, p):      
        self.nodes = []
        # Conversión de las probabilidades a espacio (0,1).
        self.probs = [(k, v/100) for k, v in sorted(p.items(), key=lambda x: x[1], reverse=True)]
        self.symbols = [s for s, _ in self.probs]
        self.codes = {}
        self.__generate_codes()
    
    def __add_node(self, sim, prob, left_node=None, right_node=None, edge_value=""):
        """
        Uso interno de la clase, añade un nodo al árbol.
        """
        self.nodes.append(self.HuffmanNode(sim, prob, left_node, right_node, edge_value))
    
    def __get_least_probable_nodes(self):
        """
        Uso interno de la clase, extrae los dos nodos con menor probabilidad.
        """
        return self.nodes.pop(-2), self.nodes.pop(-1)
    
    def __generate_codes(self):
        """
        Uso interno de la clase, generación de los códigos de Huffman para los símbolos.
        """
        if len(self.probs) <= 1:
            print("Not enough symbols, at least 2 needed")
            return
        
        for s, p in self.probs:
            self.__add_node(s, p)
        
        while len(self.nodes) > 1:
            # En cada iteración se ordenan los nodos en función de su probabilidad, orden descendente.
            self.nodes = sorted(self.nodes, key=lambda x: x.prob, reverse=True)
            # Recupera los nodos con menor probabilidad y computa símbolo y probabilidad del nuevo nodo.
            apos, lpos = self.__get_least_probable_nodes()
            nprob, nsim = apos.prob + lpos.prob, apos.simbol + lpos.simbol
            # Como los nodos están ordenados en función de su probabilidad (sentido descendiente), se asigna '1' como
            # valor de arista al nodo que primero se extrae y '0' a su siguiente (y último), que tiene la menor probabilidad.
            apos.edge_value, lpos.edge_value = "1", "0"
            # Se añade el nuevo nodo al árbol y se sigue iterando.
            self.__add_node(nsim, nprob, apos, lpos)
        
        self.__compute_codes_recursive(self.nodes[0], "")
        
    def __compute_codes_recursive(self, node, acc):
        """
        Uso interno de la clase, asignación de códigos a cada nodo hoja del árbol.
        """
        # Se propaga el código en profundidad, al empezar con valor "" pero se va añadiendo el valor de la arista de cada
        # nodo intermedio (nodo no hoja).
        acc_val = acc + node.edge_value
        
        # Si tiene nodo por la izquierda, se sigue recursivamente.
        if node.left:
            self.__compute_codes_recursive(node.left, acc_val)
        
        # Mismo proceso para el nodo derecho.
        if node.right:
            self.__compute_codes_recursive(node.right, acc_val)
        
        # Si no tiene nodo izquierdo ni derecho, se trata de un nodo hoja y se le asigna el código que se ha propagado
        # a lo largo de la recursión.
        if not node.left and not node.right:
            node.code = acc_val
            self.codes[node.simbol] = node.code
    
    def get_codes(self):
        """
        Devuelve un diccionario con los códigos para los símbolos entrados.
        """
        return {s: c for s, c in sorted(self.codes.items())}
    
    def get_compression_factor(self):
        """
        Calcula el factor de compresión.
        """
        return round(round(math.log(len(self.symbols), 2))/self.get_mean_bit_number(), 3) if len(self.probs) > 1 else "N/A"
    
    def get_mean_bit_number(self):
        """
        Calcula el promedio de bits usados.
        """
        return round(sum([len(self.codes[s])*p for s, p in self.probs]), 3) if len(self.probs) > 1 else "N/A"
    
    def get_entropy(self):
        """
        Calcula la entropía de la codificación.
        """
        return round(sum([-p*math.log(p, 2) for _, p in self.probs]), 3) if len(self.probs) > 1 else "N/A"
    
    def get_stats(self):
        return {"n_bits": self.get_mean_bit_number(), "entropy": self.get_entropy(),
               "compression_factor": self.get_compression_factor()}
            
    class HuffmanNode:
        """
        Clase de ayuda para la generación del árbol de Huffman.
        """
        
        def __init__(self, sim, prob, left_node, right_node, edge_value):
            self.simbol = sim
            self.prob = prob
            # Para los nodos hoja, left y right serán None.
            self.left = left_node
            self.right = right_node
            # El nodo ráiz no tendrá asignado ningún valor en la arista.
            self.edge_value = str(edge_value)
            # Los nodos intermedios no tendrán asignado ningún código.
            self.code = ""

In [3]:
data = {"D": 30, "K": 20, "Q": 20, "J": 15, "1": 10, "9": 5}
hcoder_a = HuffmanCoderA(data)
print(hcoder_a.get_codes())
print(hcoder_a.get_stats())

{'1': '1111', '9': '1110', 'D': '10', 'J': '110', 'K': '01', 'Q': '00'}
{'n_bits': 2.45, 'entropy': 2.409, 'compression_factor': 1.224}


### b) Modificación del apartado a) para que a partir de una tabla de símbolos originales, sea capaz de traducir al código Huffman correspondiente un mensaje cualquiera que contenga únicamente ese conjunto de símbolos originales.

<h2>[<span style="color: green"><b>Nota importante</b></span>]</h2>
<p style="font-size: 1em">El traductor a Huffman <b>SÓLO</b> funciona correctamente para símbolos de entrada completamente diferentes (ej. A, B, C, ...). En los casos en que los símbolos de entrada son similares (ej. 1, 10, 100, 00, ...) puede haber resultados erróneos.<br> Hemos intentado implementar la separación de carácteres en función de los símbolos del coder y los de la entrada, pero una vez más resulta imposible internamente en una string diferenciar si se trata de un 1, un 10 o un 100. Por esto y porque el problema se refiere a Huffman, no hemos implementado esa disciminación.</p>

In [4]:
import math
# Comentarios del apartado anterior suprimidos, solo están los nuevos del apartado b).
class HuffmanCoderB:
    
    def __init__(self, p):
        self.nodes = []
        self.probs = [(k, v/100) for k, v in sorted(p.items(), key=lambda x: x[1], reverse=True)]
        self.symbols = [s for s, _ in self.probs]
        self.codes = {}
        self.__generate_codes()
    
    def __add_node(self, sim, prob, left_node=None, right_node=None, edge_value=""):
        self.nodes.append(self.HuffmanNode(sim, prob, left_node, right_node, edge_value))
    
    def __get_least_probable_nodes(self):
        return self.nodes.pop(-2), self.nodes.pop(-1)
    
    def __generate_codes(self):
        if len(self.probs) <= 1:
            print("Not enough symbols, at least 2 needed")
            return
        for s, p in self.probs:
            self.__add_node(s, p)
        while len(self.nodes) > 1:
            self.nodes = sorted(self.nodes, key=lambda x: x.prob, reverse=True)
            apos, lpos = self.__get_least_probable_nodes()
            nprob, nsim = apos.prob + lpos.prob, apos.simbol + lpos.simbol
            apos.edge_value, lpos.edge_value = "1", "0"
            self.__add_node(nsim, nprob, apos, lpos)
        self.__compute_codes_recursive(self.nodes[0], "")
        
    def __compute_codes_recursive(self, node, acc):
        acc_val = acc + node.edge_value
        if node.left:
            self.__compute_codes_recursive(node.left, acc_val)
        if node.right:
            self.__compute_codes_recursive(node.right, acc_val)
        if not node.left and not node.right:
            node.code = acc_val
            self.codes[node.simbol] = node.code
    
    def get_codes(self):
        return {s: c for s, c in sorted(self.codes.items())}
    
    def get_compression_factor(self):
        return round(round(math.log(len(self.symbols), 2))/self.get_mean_bit_number(), 3) if len(self.probs) > 1 else "N/A"
    
    def get_mean_bit_number(self):
        return round(sum([len(self.codes[s])*p for s, p in self.probs]), 3) if len(self.probs) > 1 else "N/A"
    
    def get_entropy(self):
        return round(sum([-p*math.log(p, 2) for _, p in self.probs]), 3) if len(self.probs) > 1 else "N/A"
    
    def get_stats(self):
        return {"n_bits": self.get_mean_bit_number(), "entropy": self.get_entropy(),
               "compression_factor": self.get_compression_factor()}
    
    def translate_message(self, msg):
        """
        Traduce un mensaje a código de Huffman.
        """

        entradas = list(set([i for i in msg if i != " "]))
        # Si los símbolos del mensaje no están en el codificador no se hace nada.
        for i in entradas:
            if i not in self.symbols:
                return "Symbols do not match"

        # Se separa la string por espacios y luego por carácteres.
        split_msg, ret = [list(i) for i in msg.split(" ")], ""

        # Se remplazan los símbolos de entrada con los de Huffman, y se une el resultado primero por letra y luego por palabra.
        return " ".join(["".join(self.codes[j] for j in i) for i in split_msg])

            
    class HuffmanNode:
        
        def __init__(self, sim, prob, left_node, right_node, edge_value):
            self.simbol = sim
            self.prob = prob
            self.left = left_node
            self.right = right_node
            self.edge_value = str(edge_value)
            self.code = ""

In [5]:
data = {"A": 25, "B": 25, "C": 14, "D": 14, "E": 5.5, "F": 5.5, "G": 5.5, "H": 5.5}
hcoder_b = HuffmanCoderB(data)
print(hcoder_b.get_stats())
print(hcoder_b.translate_message("ABC DEF GH"))

{'n_bits': 2.72, 'entropy': 2.715, 'compression_factor': 1.103}
1001111 11000010000 00110010


### c) Generad ahora una secuencia larga y aleatoria que contenga los símbolos del apartado a) y codificadla en Huffmann mediante el algoritmo implementado en el apartado b).

In [6]:
from random import random

def seq_prob(fprob_table, K):
    lenght = 1
    seq_list = []
    data = ""
    while len(seq_list) < lenght:
        for k, v in fprob_table.items():
            if len(data) == K:
                seq_list.append(data)
                data = ""
                break
            #Genera un numero random i si este es menor a la probabilidad del simbolo se añade
            rn = random()
            if rn <=  (v/100):
                data += k

    return seq_list


In [7]:
data = {"D": 30, "K": 20, "Q": 20, "J": 15, "1": 10, "9": 5}
seq = seq_prob(data, 250)
print('Secuencia: ', seq, "\n")
hcoder_c = HuffmanCoderB(data)
print(hcoder_c.get_codes(), "\n")
print(hcoder_c.translate_message(seq[0]))

Secuencia:  ['DDKQKQDKQJDK9Q1DKQQQJKQD11DJ1K19KJ1QJKQQKQDKDDKDKJJDJKDQ1DQ1K1QDDJDQ1QQKQDQJDQKQDDJKQKJ1QQD1DKJQ1Q1D1DKQ1DQDQD9KDQDQDDJ1KQ1DQJQDJDDD11DQJQKJKQJDKDJKDKDJDDKJKD1D1DQKQ9D1DKJ1KK1DDD9KDDJK1JDJK1J19Q1DJDKKQDDJ1JDKDDK9DDJ9KJKKQD1DKQK1D1D1DKJDKDQDJ9Q11QJD1K'] 

{'1': '1111', '9': '1110', 'D': '10', 'J': '110', 'K': '01', 'Q': '00'} 

101001000100100100110100111100011111001000000110010010111111111011011110111111110011101111001100100000100100110100110011101101011001100011111000111101111100101011010001111000001001000110100001001010110010001110111100001011111001110001111001111101111100100111110001000101110011000100010101101111010011111000110001011010101011111111100011000011100100110100110110011001101101010011100110111110111110000100111010111110011101111010111111010101110011010110011111110101100111111101111111000111110110100101001010110111111010011010011110101011011100111001010010111110010001111110111110111110011101001100010110111000111111110011010111101


### d) Ejecutad el programa con diversas secuencias aleatorias, suficientemente largas. ¿Qué factor de factor de compresión alcanzáis? ¿Qué relación tiene éste con la entropía calculada teóricamente para el conjunto de símbolos del apartado a)?

In [9]:
def seq_n_times(fprob_table, K, N):
    seq_list_n = []
    for i in range(N):
        seq_list_n.append(seq_prob(fprob_table, K))
    return seq_list_n[-1]

In [10]:
data = {"D": 30, "K": 20, "Q": 20, "J": 15, "1": 10, "9": 5}
seq = seq_n_times(data, 250, 50)
print('Secuencia: ', seq, "\n")
hcoder_d = HuffmanCoderB(data)
print(hcoder_d.get_codes(), "\n")
print(hcoder_d.translate_message(seq[0]), "\n")
print(hcoder_d.get_stats(), "\n")

Secuencia:  ['Q99KKDDKDQJQKQKD1QDKJDKJDQDKQQJDQ19DD1Q1DJDQD1QDJJDJDDKJDKDQKQKDDQQJQJQ9DDKQJ1DK1KK1DQDDKD1DKQKDDDQKQ1DQDKQKQ1JDKDDDKDKQ19D1Q1DKJQKJ1DDDJDKDKQ1D9KQ1DQK9DKQDQDDJ1KDDK9JDKDJKDQKD1D1KKDKDDDDQQ1QJ11DDKDD1DD9KQKDDKQDKQJDKJQDK1DJDJ9DJJ1DKDJDKQKQ1DQJ1DQJKK9'] 

{'1': '1111', '9': '1110', 'D': '10', 'J': '110', 'K': '01', 'Q': '00'} 

0011101110010110100110001100001000110111100100111010011101000100100001101000111111101010111100111110110100010111100101101101011010100111010011000010001101000001100011000111010100100110111110011111010111111000101001101111100100011010100001001111100010010001001111110100110101001100100111111101011110011111001110000111011111010101101001100100111110111001001111100001111010010010001010110111101101001111011010011011001100001101111101111010110011010101000001111001101111111110100110101111101011100100011010010010010011010011100010011111101101011011101011011011111001101101001000100111110001101111100011001011110 

{'n_bits': 2.45, 'entropy': 2.409, 'compression

##### ¿Qué factor de factor de compresión alcanzáis?
 2.409
 
##### ¿Qué relación tiene éste con la entropía calculada teóricamente para el conjunto de símbolos del apartado a)
Teniendo en cuenta que la entropía es la media de la cantidad de información contenida en una secuencia de entrada y tras varias pruebas, hemos podido observar que a mayor ratio de compresión se obtiene una entropía menor.